In [1]:
import json
import requests
import os
from dotenv import load_dotenv
import pandas as pd
from pandas import json_normalize 


load_dotenv(verbose=True)

True

In [38]:
api_key = os.getenv("CRUNCHBASE_API_KEY") # extract API key 
base_url = "https://api.crunchbase.com/api/"
params = {"user_key" : api_key}

**API notes**:

- If we do specify any limit, the limit is set to 50 items

## Simple data extraction without filtering

In [76]:
query = {
    "field_ids": [
        "name",
        "location_identifiers",
        # "categories",
        # "website",
        # "linkedin",
        "listed_stock_symbol",
        "location_identifiers",
        "num_acquisitions",
        # "num_articles",
        # "num_current_positions",
        "status"
    ],
    "limit": 100,
}

In [77]:
r = requests.post(f"https://api.crunchbase.com/api/v4/searches/organizations", json=query, params=params)

In [78]:
json_data = json.loads(r.text)
json_data

{'count': 1280126,
 'entities': [{'uuid': '1a410398-3a72-5882-99b8-6318cf594850',
   'properties': {'name': 'SoftBank',
    'identifier': {'permalink': 'softbank',
     'image_id': 'v1476460859/u55msztjdjwmwroalw6c.jpg',
     'uuid': '1a410398-3a72-5882-99b8-6318cf594850',
     'entity_def_id': 'organization',
     'value': 'SoftBank'},
    'num_acquisitions': 12,
    'status': 'ipo',
    'location_identifiers': [{'permalink': 'tokyo-tokyo',
      'uuid': '127ba62d-d40a-3d12-42c0-e606f9fa9440',
      'location_type': 'city',
      'entity_def_id': 'location',
      'value': 'Tokyo'},
     {'permalink': 'tokyo-japan',
      'uuid': 'ee6ee543-afd4-5595-8c2b-ffd3ea11a243',
      'location_type': 'region',
      'entity_def_id': 'location',
      'value': 'Tokyo'},
     {'permalink': 'japan',
      'uuid': 'e2781ac6-611b-376a-fc7d-ddec670c3d94',
      'location_type': 'country',
      'entity_def_id': 'location',
      'value': 'Japan'},
     {'permalink': 'asia',
      'uuid': 'b41404d2-8

In [79]:
df = pd.json_normalize(json_data['entities'])
df = df.drop("uuid", axis=1)
df.head()

,properties.name,properties.identifier.permalink,properties.identifier.image_id,properties.identifier.uuid,properties.identifier.entity_def_id,properties.identifier.value,properties.num_acquisitions,properties.status,properties.location_identifiers,properties.listed_stock_symbol
0,SoftBank,softbank,v1476460859/u55msztjdjwmwroalw6c.jpg,1a410398-3a72-5882-99b8-6318cf594850,organization,SoftBank,12.0,ipo,"[{'permalink': 'tokyo-tokyo', 'uuid': '127ba62...",NaN
1,Reddit,reddit,v1481123587/dhr7ibainyczvxprrsax.png,0954d5cd-96d1-9017-901a-0b28284f06f7,organization,Reddit,3.0,was_acquired,"[{'permalink': 'san-francisco-california', 'uu...",NaN
2,Crowdcube,crowdcube,smdxzio4kigyx52cnlpu,01a2a666-577c-1673-eba6-74c02aaf2e80,organization,Crowdcube,1.0,operating,"[{'permalink': 'exeter-devon', 'uuid': 'c5f8ba...",NaN
3,Calendly,calendly,v1467257042/ysmcpzp0tkc0ixsdrhvp.png,2f6131ca-1a42-5894-6195-a55053fff7a1,organization,Calendly,NaN,operating,"[{'permalink': 'atlanta-georgia', 'uuid': 'd80...",NaN
4,Ant Group,ant-group,sgzmygfsbqgmbxslqreb,74a20af3-f4dd-6188-de60-c4ee6cd0ca4a,organization,Ant Group,5.0,operating,"[{'permalink': 'hangzhou-zhejiang', 'uuid': '4...",NaN


### Add filter: only companies whose number of articles are 110

In [7]:
query1 = {
    "field_ids": [
        "name",
        "location_identifiers",
        "listed_stock_symbol",
        "location_identifiers",
        "num_acquisitions",
        "status",
        "num_articles",
    ],
    "limit": 10,
    "query": [
        {
          "type": "predicate",
          "field_id": "num_articles",
          "operator_id": "eq",
          "values": [110]
        },
    ]
    
}

In [8]:
r = requests.post(f"https://api.crunchbase.com/api/v4/searches/organizations", json=query1, params=params)

In [9]:
json_data = json.loads(r.text)
# json_data

In [10]:
df = pd.json_normalize(json_data['entities'])
df = df.drop("uuid", axis=1)
df.head()

,properties.name,properties.identifier.permalink,properties.identifier.image_id,properties.identifier.uuid,properties.identifier.entity_def_id,properties.identifier.value,properties.status,properties.location_identifiers,properties.num_articles,properties.num_acquisitions
0,Algorand,algorand,y1vj0utwbwnvtqvebpl9,12367b55-7b07-4ca6-a902-258d5ff6bac8,organization,Algorand,operating,"[{'permalink': 'boston-massachusetts', 'uuid':...",110,NaN
1,TrustRadius,trustradius,v1505336761/qwd6i0uw0hwwmf83wk89.jpg,ceee2aca-e2ae-156d-33aa-6d00b666e17d,organization,TrustRadius,operating,"[{'permalink': 'austin-texas', 'uuid': '05f9ce...",110,NaN
2,aCommerce,acommerce,frtgksxtrnugwctukguo,5590fd14-c17a-9d39-b43c-157fd3537076,organization,aCommerce,operating,"[{'permalink': 'bangkok-krung-thep', 'uuid': '...",110,NaN
3,StyleSeat,styleseat,s1kveuruswfmvwdramgn,2f83b3a0-32ca-da50-f476-3c642df8d663,organization,StyleSeat,operating,"[{'permalink': 'san-francisco-california', 'uu...",110,1.0
4,Behalf,behalf,v1473241962/jvrhg9qcxzo5ablujd0u.png,fae6249a-80d4-86d6-0a31-5768ea19e6d6,organization,Behalf,operating,"[{'permalink': 'new-york-new-york', 'uuid': 'd...",110,NaN


### Add filter: only companies whose number of articles are 110 and still operating 

In [39]:
query1 = {
    "field_ids": [
        "name",
        "location_identifiers",
        "listed_stock_symbol",
        "location_identifiers",
        "num_acquisitions",
        "status",
        "num_articles",
    ],
    "limit": 10,
    "query": [
        {
          "type": "predicate",
          "field_id": "num_articles",
          "operator_id": "eq",
          "values": [110]
        },
        {
          "type": "predicate",
          "field_id": "status",
          "operator_id": "eq",
          "values": ["operating"]
        },
    ]
    
}

In [40]:
r = requests.post(f"https://api.crunchbase.com/api/v4/searches/organizations", json=query1, params=params)

In [41]:
json_data = json.loads(r.text)
# json_data

In [42]:
df = pd.json_normalize(json_data['entities'])
df = df.drop("uuid", axis=1)
df.head()

,properties.name,properties.identifier.permalink,properties.identifier.image_id,properties.identifier.uuid,properties.identifier.entity_def_id,properties.identifier.value,properties.status,properties.location_identifiers,properties.num_articles,properties.num_acquisitions
0,Algorand,algorand,y1vj0utwbwnvtqvebpl9,12367b55-7b07-4ca6-a902-258d5ff6bac8,organization,Algorand,operating,"[{'permalink': 'boston-massachusetts', 'uuid':...",110,NaN
1,TrustRadius,trustradius,v1505336761/qwd6i0uw0hwwmf83wk89.jpg,ceee2aca-e2ae-156d-33aa-6d00b666e17d,organization,TrustRadius,operating,"[{'permalink': 'austin-texas', 'uuid': '05f9ce...",110,NaN
2,aCommerce,acommerce,frtgksxtrnugwctukguo,5590fd14-c17a-9d39-b43c-157fd3537076,organization,aCommerce,operating,"[{'permalink': 'bangkok-krung-thep', 'uuid': '...",110,NaN
3,StyleSeat,styleseat,s1kveuruswfmvwdramgn,2f83b3a0-32ca-da50-f476-3c642df8d663,organization,StyleSeat,operating,"[{'permalink': 'san-francisco-california', 'uu...",110,1.0
4,Behalf,behalf,v1473241962/jvrhg9qcxzo5ablujd0u.png,fae6249a-80d4-86d6-0a31-5768ea19e6d6,organization,Behalf,operating,"[{'permalink': 'new-york-new-york', 'uuid': 'd...",110,NaN


### Add filter: only companies in Switzerland

In [61]:
query3 = {
    "field_ids": [
        "name",
        "created_at",
        "description",
        "location_identifiers",
        "funding_stage",
        "funding_total",
    ],
    "limit": 1000,
    "query": [
        {
          "type": "predicate",
          "field_id": "location_identifiers",
          "operator_id": "includes",
          "values": ["Switzerland"]
        },
    ]
}

In [62]:
r = requests.post(f"https://api.crunchbase.com/api/v4/searches/organizations", json=query3, params=params)

In [63]:
json_data = json.loads(r.text)
json_data

{'count': 16108,
 'entities': [{'uuid': '96a3355d-8f90-48ab-7354-36335f71adce',
   'properties': {'name': 'Partners Group',
    'identifier': {'permalink': 'partners-group',
     'image_id': 'v1397179710/a323dcf35bfce57511f2f37efb8dfa87.gif',
     'uuid': '96a3355d-8f90-48ab-7354-36335f71adce',
     'entity_def_id': 'organization',
     'value': 'Partners Group'},
    'description': 'Partners Group are a unique, global alternative asset manager with strong Swiss roots, proactively providing their international client base with innovative, customized solutions.\r\n\r\nWith CHF 19.2 billion in private equity assets under management, a global team of over 130 private equity professionals in seven offices and over 300 partnership investments led by over 200 general partners, Partners Group is one of the largest private equity asset managers worldwide.',
    'funding_total': {'value_usd': 500000000,
     'currency': 'USD',
     'value': 500000000},
    'created_at': '2009-01-04T00:00:23Z',


In [64]:
df = pd.json_normalize(json_data['entities'])
df = df.drop(["uuid", "properties.identifier.image_id", "properties.identifier.uuid"], axis=1)
df.head()

,properties.name,properties.identifier.permalink,properties.identifier.entity_def_id,properties.identifier.value,properties.description,properties.funding_total.value_usd,properties.funding_total.currency,properties.funding_total.value,properties.created_at,properties.location_identifiers,properties.funding_stage
0,Partners Group,partners-group,organization,Partners Group,"Partners Group are a unique, global alternativ...",500000000.0,USD,500000000.0,2009-01-04T00:00:23Z,"[{'permalink': 'baar-zug', 'uuid': '161b7401-4...",ipo
1,Novartis,novartis,organization,Novartis,Novartis is a global healthcare company based ...,6500000.0,USD,6500000.0,2008-12-24T20:27:57Z,"[{'permalink': 'basel-basel-stadt', 'uuid': '6...",ipo
2,Polkadot,polkadot,organization,Polkadot,Polkadot is a protocol that allows independent...,188700000.0,USD,188700000.0,2017-10-16T04:28:22Z,"[{'permalink': 'zug-zug', 'uuid': '2374ed28-a8...",NaN
3,ConsenSys,consensus-systems,organization,ConsenSys,ConsenSys is a market leading blockchain techn...,10000000.0,USD,10000000.0,2015-06-12T15:38:04Z,"[{'permalink': 'zug-zug', 'uuid': '2374ed28-a8...",NaN
4,Climeworks,climeworks,organization,Climeworks,Climeworks captures CO2 from air via commercia...,138733320.0,CHF,133630000.0,2014-04-28T12:29:02Z,"[{'permalink': 'zürich-zurich', 'uuid': 'bd22e...",NaN


In [65]:
list(df["properties.location_identifiers"])

[[{'permalink': 'baar-zug',
   'uuid': '161b7401-4fa2-4e92-c7e6-1524cc0dda7e',
   'location_type': 'city',
   'entity_def_id': 'location',
   'value': 'Baar'},
  {'permalink': 'zug-switzerland',
   'uuid': 'efdf04ff-fcb7-742c-ad0d-b2be1dae0683',
   'location_type': 'region',
   'entity_def_id': 'location',
   'value': 'Zug'},
  {'permalink': 'switzerland',
   'uuid': '078d9679-a862-02a2-57c8-8337e9a1eec8',
   'location_type': 'country',
   'entity_def_id': 'location',
   'value': 'Switzerland'},
  {'permalink': 'europe',
   'uuid': '6106f5dc-823e-5da8-40d7-51612c0b2c4e',
   'location_type': 'continent',
   'entity_def_id': 'location',
   'value': 'Europe'}],
 [{'permalink': 'basel-basel-stadt',
   'uuid': '6da0d74a-7033-c89e-4ae3-974e54a2eaed',
   'location_type': 'city',
   'entity_def_id': 'location',
   'value': 'Basel'},
  {'permalink': 'basel-stadt-switzerland',
   'uuid': '6664754b-543f-6588-9266-2f4e4ae56a58',
   'location_type': 'region',
   'entity_def_id': 'location',
   'val

## People Dataset (same logic)

In [25]:
query_people = {
    "field_ids": [
        "investor_stage",
        "linkedin",
        "location_identifiers",
        "num_articles"
    ],
    "limit": 10,
}

In [26]:
r = requests.post(f"https://api.crunchbase.com/api/v4/searches/people", json=query_people, params=params)

In [27]:
json_data = json.loads(r.text)
# json_data

In [28]:
df = pd.json_normalize(json_data['entities'])
df = df.drop("uuid", axis=1)
df.head()

,properties.identifier.permalink,properties.identifier.image_id,properties.identifier.uuid,properties.identifier.entity_def_id,properties.identifier.value,properties.investor_stage,properties.linkedin.value,properties.location_identifiers,properties.num_articles
0,bill-gates,v1448909339/owy8vzc7yerywuljcna0.jpg,62ab53b0-3a28-0cb7-9995-fd3215b78e41,person,Bill Gates,[seed],https://www.linkedin.com/in/williamhgates,"[{'permalink': 'seattle-washington', 'uuid': '...",4385
1,kunal-shah-2,pnxnrhtqaw7jxsti0qqx,7425ae5a-3f37-467f-b9d2-9511d89d2b55,person,Kunal Shah,[initial_coin_offering],NaN,"[{'permalink': 'mumbai-maharashtra', 'uuid': '...",110
2,tim-kendall,yzky7jb7an8uf66y6wcr,256ac5b8-d529-0780-8c78-80f38a459074,person,Tim Kendall,[seed],http://www.linkedin.com/pub/tim-kendall/0/2/657,"[{'permalink': 'san-francisco-california', 'uu...",113
3,mark-zuckerberg,v1448830269/gzcifut4c2xah95x0ewd.jpg,a01b8d46-d311-3333-7c34-aa3ae9c03f22,person,Mark Zuckerberg,[seed],NaN,"[{'permalink': 'palo-alto-california', 'uuid':...",3986
4,justin-mateen,v1397634299/cef9ad8ffa025ae830bd7cbf089c976e.jpg,1601ab0b-87bb-5ee5-9545-c289d4f706cf,person,Justin Mateen,"[convertible_note, early_stage_venture, seed, ...",http://www.linkedin.com/pub/justin-mateen/6a/a...,"[{'permalink': 'united-states', 'uuid': 'f110f...",98


In [29]:
df["properties.location_identifiers"]

0    [{'permalink': 'seattle-washington', 'uuid': '...
1    [{'permalink': 'mumbai-maharashtra', 'uuid': '...
2    [{'permalink': 'san-francisco-california', 'uu...
3    [{'permalink': 'palo-alto-california', 'uuid':...
4    [{'permalink': 'united-states', 'uuid': 'f110f...
5    [{'permalink': 'san-francisco-california', 'uu...
6    [{'permalink': 'tehran-tehran', 'uuid': '77f17...
7    [{'permalink': 'san-francisco-california', 'uu...
8    [{'permalink': 'san-francisco-california', 'uu...
9    [{'permalink': 'greater-seattle-area', 'uuid':...
Name: properties.location_identifiers, dtype: object